In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import ast
import networkx as nx
from pyvis.network import Network
#import graphviz
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('tableau-colorblind10')
from torch_geometric.explain.metric import groundtruth_metrics
import torch
import numba
import seaborn as sns
import iqplot
from sklearn import metrics
import bokeh.models
import bokeh.plotting
import bokeh.io
bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
df = pd.read_csv('data/TexasBetaExplainerGraphProbs.csv')
df = df[df['seed'] == 90820]
df.head()

,Unnamed: 0,seed,prob,p1,p2,class 1,class 2
1300,1300,90820,0.631084,0,58,3,2
1301,1301,90820,0.739530,0,121,3,0
1302,1302,90820,0.353611,1,80,0,4
1303,1303,90820,0.621147,2,8,2,3
1304,1304,90820,0.604771,4,66,4,0


In [3]:
df

,Unnamed: 0,seed,prob,p1,p2,class 1,class 2
1300,1300,90820,0.631084,0,58,3,2
1301,1301,90820,0.739530,0,121,3,0
1302,1302,90820,0.353611,1,80,0,4
1303,1303,90820,0.621147,2,8,2,3
1304,1304,90820,0.604771,4,66,4,0
...,...,...,...,...,...,...,...
1620,1620,90820,0.324962,180,47,4,2
1621,1621,90820,0.410325,180,66,4,0
1622,1622,90820,0.389563,180,84,4,2
1623,1623,90820,0.398591,180,180,4,4


In [4]:
subset = df[df['prob'] >= 0.5]
print(subset.shape, df.shape, 157/325)

(157, 7) (325, 7) 0.48307692307692307


In [5]:
color = dict()
color[0] = '#2E2585'
color[1] = '#337538'
color[2] = '#5DA899'
color[3] = '#94CBEC'
color[4] = '#DCCD7D'
node2class = dict()
c1 = list(df['class 1'])
n1 = list(df['p1'])
for i in range(0, len(n1)):
    if n1[i] not in list(node2class.keys()):
        node2class[n1[i]] = c1[n1[i]]
c1 = list(df['class 2'])
n1 = list(df['p2'])
for i in range(0, len(n1)):
    if n1[i] not in list(node2class.keys()):
        node2class[n1[i]] = c1[n1[i]]

In [6]:
g = Network(notebook=True, directed=True)
ei1 = list(df['p1'])
ei2 = list(df['p2'])
prob = list(df['prob'])
b1 = []
b2 = []
n = []
c = []
width = dict()
wtr = set()
lst = []
for i in range(len(ei1)):
    if prob[i] >= 0.5:
        b1.append(ei1[i])
        b2.append(ei2[i])
        lst.append(prob[i])
        if (ei1[i], ei2[i]) and (ei2[i], ei1[i]) not in wtr:
            if ei1[i] in width.keys():
                width[ei1[i]][ei2[i]] = prob[i]
            else:
                width[ei1[i]] = {ei2[i]: prob[i]}
            if ei2[i] in width.keys():
                width[ei2[i]][ei1[i]] = prob[i]
            else:
                width[ei2[i]] = {ei1[i]: prob[i]}
            wtr.add((ei1[i], ei2[i]))
            wtr.add((ei2[i], ei1[i]))
u1 = np.unique(b1)
u2 = np.unique(b2)
for i in u1:
    if i not in n:
        n.append(i)
        c.append(color[node2class[i]])
for i in u2:
    if i not in n:
        n.append(i)
        c.append(color[node2class[i]])
g.add_nodes(n, color=c)
mx = np.max(lst)
mn = np.min(lst)
for i in range(0, len(b1)):
    w = 5 * (width[b1[i]][b2[i]] - mn + 1e-5) / (mx - mn)
    if w <= 0:
        print(w)
    g.add_edge(b1[i], b2[i], width=w)
for edge in g.get_edges():
    p1 = edge['from']
    p2 = edge['to']
    w = width[b1[i]][b2[i]]
    edge['color'] = '#000000'
g.show('data/Vis/TexasGraphBetaProbs.html')

data/Vis/TexasGraphBetaProbs.html


In [7]:
df = df.rename({'prob': 'Probability'}, axis=1)
p = iqplot.ecdf(data=df, q="Probability")
bokeh.io.show(p)

In [8]:
g = Network(notebook=True, directed=True)
ei1 = list(df['p1'])
ei2 = list(df['p2'])
b1 = []
b2 = []
n = []
c = []
width = dict()
wtr = set()
lst = []
for i in range(len(ei1)):
    b1.append(ei1[i])
    b2.append(ei2[i])
u1 = np.unique(b1)
u2 = np.unique(b2)
for i in u1:
    if i not in n:
        n.append(i)
        c.append(color[node2class[i]])
for i in u2:
    if i not in n:
        n.append(i)
        c.append(color[node2class[i]])
g.add_nodes(n, color=c)
for i in range(0, len(b1)):
    w = 5
    if w <= 0:
        print(w)
    g.add_edge(b1[i], b2[i], width=w)
for edge in g.get_edges():
    p1 = edge['from']
    p2 = edge['to']
    w = 5
    edge['color'] = '#000000'
g.show('data/Vis/TexasGraphOriginal.html')

data/Vis/TexasGraphOriginal.html


In [26]:
df = pd.read_csv('data/TexasSubgraphXGraphProbs.csv')
df = df[df['seed'] == 27275]
df.head()

,seed,prob,p1,p2,class 1,class 2
325,27275,0,0,58,3,2
326,27275,0,0,121,3,0
327,27275,0,1,80,0,4
328,27275,0,2,8,2,3
329,27275,0,4,66,4,0


In [28]:
g = Network(notebook=True, directed=True)
ei1 = list(df['p1'])
ei2 = list(df['p2'])
prob = list(df['prob'])
b1 = []
b2 = []
n = []
c = []
width = dict()
wtr = set()
lst = []
for i in range(len(ei1)):
    if prob[i] >= 0.5:
        b1.append(ei1[i])
        b2.append(ei2[i])
        lst.append(prob[i])
u1 = np.unique(b1)
u2 = np.unique(b2)
for i in u1:
    if i not in n:
        n.append(i)
        c.append(color[node2class[i]])
for i in u2:
    if i not in n:
        n.append(i)
        c.append(color[node2class[i]])
g.add_nodes(n, color=c)
mx = np.max(lst)
mn = np.min(lst)
for i in range(0, len(b1)):
    w = 5
    g.add_edge(b1[i], b2[i], width=w)
for edge in g.get_edges():
    p1 = edge['from']
    p2 = edge['to']
    w = 5
    edge['color'] = '#000000'
g.show('data/Vis/TexasGraphSubgraphXProbs.html')

data/Vis/TexasGraphSubgraphXProbs.html


In [11]:
df = pd.read_csv('data/TexasGNNExplainerGraphProbs.csv')
df = df[df['seed'] == 68268]
df.head()

,Unnamed: 0,seed,prob,p1,p2,class 1,class 2
325,325,68268,0.000080,0,58,3,2
326,326,68268,0.999949,0,121,3,0
327,327,68268,0.000085,1,80,0,4
328,328,68268,0.000062,2,8,2,3
329,329,68268,0.000097,4,66,4,0


In [12]:
g = Network(notebook=True, directed=True)
ei1 = list(df['p1'])
ei2 = list(df['p2'])
prob = list(df['prob'])
b1 = []
b2 = []
n = []
c = []
width = dict()
wtr = set()
lst = []
for i in range(len(ei1)):
    if prob[i] >= 0.5:
        b1.append(ei1[i])
        b2.append(ei2[i])
        lst.append(prob[i])
u1 = np.unique(b1)
u2 = np.unique(b2)
for i in u1:
    if i not in n:
        n.append(i)
        c.append(color[node2class[i]])
for i in u2:
    if i not in n:
        n.append(i)
        c.append(color[node2class[i]])
g.add_nodes(n, color=c)
mx = np.max(lst)
mn = np.min(lst)
for i in range(0, len(b1)):
    w = 5
    g.add_edge(b1[i], b2[i], width=w)
for edge in g.get_edges():
    p1 = edge['from']
    p2 = edge['to']
    edge['color'] = '#000000'
g.show('data/Vis/TexasGraphSubgraphXProbs.html')

data/Vis/TexasGraphSubgraphXProbs.html
